# Practica 13
- **Alumna:** Enya Quetzalli Gómez Rodríguez *(Eduardo Gómez Rodríguez)*
- **Profesora:** Olga Kolesnikova
- **Escuela:** Escuela Superior de Cómputo del IPN
- **Grupo:** 3CV9
- **Semestre:** 2020/2

## Obtener las palabras mas frecuentes
**Instrucciones:**
    Obtener una lista de las palabras más frecuentes en el documento y ordenarlas en forma descendente.

In [1]:
import nltk
import numpy
import math
from nltk.stem import SnowballStemmer

In [2]:
def cleanHtml(txt):
	from bs4 import BeautifulSoup
	return BeautifulSoup(txt,'lxml').get_text().lower()

In [3]:
def splitText(txt):
	return txt.replace('/', ' ').replace('.', ' ').replace('-', ' ')

In [4]:
def deleteTrash(txt):
	import re
	good = {'\n'}
	for i in "abcdefghijklmnopqrstuvwxyz áéíóúñü":
		good.add(i)
	ans = ""
	for c in txt:
		if c in good:
			ans += c
	return ans

In [5]:
def splitToSentences(txt):
    tokenizer = nltk.data.load('nltk:tokenizers/punkt/english.pickle')
    return tokenizer.tokenize(txt)

In [6]:
def deleteStopwords(txt):
	from nltk.corpus import stopwords
	ans = []
	stp = stopwords.words()
	for w in txt:
		if w not in stp:
			ans.append(w)
	return ans

In [7]:
def getVocabulary(sentences):
    tokens = set()
    for sent in sentences:
        for token in sent:
            tokens.add(token)
    return sorted(tokens)

In [8]:
def getFrequencies(sentences, vocabulary):
    tokens = dict()
    
    for token in vocabulary:
        tokens[token] = 0
    
    for sent in sentences:
        for token in sent:
            tokens[token] += 1
            
    return tokens

In [9]:
def getContexts(wordList, windowSize=4):
    contexts = dict()
    index = 0
    for index in range(len(wordList)):
        word = wordList[index]
        if word not in contexts:
            contexts[word] = []
        start = max(0, index - windowSize)
        end = min(index + windowSize, len(wordList) - 1)
        for i in range(start, end + 1):
            if i != index:
                contexts[word].append(wordList[i])
    return contexts

In [10]:
def loadFromJson(name):
    import json
    with open("./files/"+name, 'r', encoding="utf8") as f:
        return json.load(f)

In [11]:
def dumpToJson(obj, name):
    import json
    with open("./files/"+name, 'w', encoding="utf8") as outfile:
        json.dump(obj, outfile, indent=4)

In [12]:
def replaceWithStems(tokens):
    ss = SnowballStemmer("spanish")
    stematized_tokens = []
    for word in tokens:
        stematized_tokens.append(ss.stem(word))
    return stematized_tokens

In [13]:
def replacWithLemmas(tokens):
    lemmas = loadFromJson("lemmatized_tokens_generate")
    lemmatized_tokens = []
    for word in tokens:
        if word in lemmas.keys():
            lemmatized_tokens.append(lemmas[word])
        else:
            lemmatized_tokens.append(word)
    return lemmatized_tokens

In [14]:
def getTagger():
    patterns = [
        (r'.*o$', 'n'),  # noun masculine singular
        (r'.*os$', 'n'), # noun masculine plural
        (r'.*a$', 'n'),  # noun femenine singular
        (r'.*as$', 'n')  # noun femenine plural
    ]
    regexTagger = nltk.RegexpTagger(patterns, nltk.DefaultTagger('s'))
    unigramTagger = nltk.UnigramTagger(nltk.corpus.cess_esp.tagged_sents(), None, regexTagger)
    return unigramTagger

In [15]:
def mixTags(tokens, tokenTags):
    taggedTokens = list()
    for i in range(0,len(tokens)):
        taggedTokens.append((tokens[i], tokenTags[i][1]))
    return taggedTokens

In [16]:
def normalizeSencences(sent):
    tagger = getTagger()
    sentences = []
    for s in sent:
        cleanSentence = deleteTrash(splitText(s))
        normalizedTokens = deleteStopwords(nltk.word_tokenize(cleanSentence))
        tokenTags = tagger.tag(normalizedTokens)
        stematizedTokens = replaceWithStems(normalizedTokens)
        tokens = mixTags(stematizedTokens, tokenTags)
        sentences.append(tokens)
    return sentences

In [17]:
f = open("./files/e961024.htm", "r", encoding="utf8")
org_txt = f.read()
f.close()

sentences = splitToSentences(cleanHtml(org_txt))
sentences = normalizeSencences(sentences)

In [18]:
vocabulary = getVocabulary(sentences)
frequencies = getFrequencies(sentences, vocabulary)

In [20]:
commonWord = sorted([(freq,token) for token,freq in frequencies.items()], reverse=True)
for freq,token in commonWord:
    print(str(freq), str(token))

186 ('mexic', 'np0000l')
161 ('millon', 'ncmp000')
158 ('excelsior', 's')
112 ('octubr', 'W')
105 ('merc', 'ncms000')
99 ('pais', 'ncms000')
98 ('pes', 'Zm')
94 ('año', 'ncms000')
89 ('cient', 'n')
88 ('gobiern', 'ncms000')
87 ('part', 'ncfs000')
81 ('sol', 'rg')
79 ('juev', 'W')
77 ('dolar', 'Zm')
75 ('www', 's')
75 ('mx', 's')
75 ('http', 's')
75 ('html', 's')
74 ('siguient', 'aq0cs0')
72 ('not', 'ncfs000')
70 ('ser', 'vsn0000')
69 ('nacional', 'aq0cs0')
64 ('financ', 'aq0fs0')
60 ('president', 'ncms000')
59 ('años', 'ncmp000')
57 ('mil', 'dn0cp0')
55 ('agu', 'ncfs000')
51 ('fin', 'ncms000')
50 ('petroquim', 'n')
49 ('sin', 'cc')
49 ('empres', 'ncfp000')
48 ('sector', 'ncms000')
48 ('primer', 'ao0fs0')
48 ('iglesi', 'ncfs000')
46 ('cambi', 'ncms000')
46 ('asi', 'rg')
44 ('hoy', 'rg')
44 ('banc', 'ncms000')
43 ('vez', 'ncfs000')
43 ('polit', 'ncfs000')
43 ('embarg', 'ncms000')
42 ('ahor', 'rg')
41 ('prim', 'ao0ms0')
41 ('mexican', 'aq0ms0')
41 ('dij', 'vmis3s0')
41 ('dec', 'vmn0000')


10 ('represent', 'nccs000')
10 ('relacion', 'ncfp000')
10 ('razon', 'ncfs000')
10 ('proteccion', 'ncfs000')
10 ('propi', 'di0ms0')
10 ('practic', 'rg')
10 ('posibil', 'ncfs000')
10 ('opinion', 'ncfs000')
10 ('oper', 'ncfs000')
10 ('ofrec', 'vmip3s0')
10 ('ocurr', 'vmis3s0')
10 ('new', 's')
10 ('mism', 'di0mp0')
10 ('logr', 'vmn0000')
10 ('ley', 'ncfp000')
10 ('inversion', 'ncfp000')
10 ('intervencion', 'ncfs000')
10 ('intern', 'aq0fs0')
10 ('imss', 's')
10 ('hombr', 'ncms000')
10 ('gonzalez', 's')
10 ('gent', 'ncfs000')
10 ('funcionari', 'ncmp000')
10 ('fuert', 'aq0cs0')
10 ('evit', 'vmn0000')
10 ('esfuerz', 'ncms000')
10 ('epoc', 'ncfs000')
10 ('emisor', 'n')
10 ('elev', 'vmn0000')
10 ('efect', 'ncmp000')
10 ('dye', 's')
10 ('dictam', 'ncms000')
10 ('dej', 'vmn0000')
10 ('deb', 'vmp00sm')
10 ('dcr', 's')
10 ('cotiz', 's')
10 ('contrari', 'ncms000')
10 ('consum', 'ncms000')
10 ('conflict', 'ncms000')
10 ('central', 'aq0cs0')
10 ('cas', 'ncfp000')
10 ('caract', 'ncms000')
10 ('boletin',

5 ('repart', 'ncms000')
5 ('rentabil', 'ncfs000')
5 ('religi', 'aq0fs0')
5 ('reiter', 'vmis3s0')
5 ('reflexion', 'ncfs000')
5 ('red', 'ncfs000')
5 ('reconoc', 'vmip3s0')
5 ('reconoc', 'ncms000')
5 ('recient', 'rg')
5 ('rebolled', 'n')
5 ('realiz', 'vmn0000')
5 ('realiz', 'vmis3s0')
5 ('razon', 's')
5 ('radiofrecuent', 'n')
5 ('quier', 'vmip3p0')
5 ('qued', 'vmn0000')
5 ('puest', 'ncmp000')
5 ('puest', 'ncfs000')
5 ('puert', 'ncms000')
5 ('public', 'aq0msp')
5 ('prueb', 'ncfs000')
5 ('propuest', 'ncfs000')
5 ('proporcion', 'vmn0000')
5 ('privatiz', 'n')
5 ('prestam', 'ncms000')
5 ('presion', 'ncfs000')
5 ('pregunt', 'ncfs000')
5 ('posterior', 'rg')
5 ('posibil', 'ncfp000')
5 ('porcentual', 's')
5 ('pon', 'vmn0000')
5 ('polic', 'nccs000')
5 ('pobr', 'aq0cp0')
5 ('plat', 'ncfs000')
5 ('plant', 'ncfp000')
5 ('plan', 'ncmp000')
5 ('pierd', 'vmip3s0')
5 ('peñaloz', 'n')
5 ('permit', 'vmip3s0')
5 ('pas', 'vmn0000')
5 ('particip', 'nccp000')
5 ('paraestatal', 's')
5 ('papel', 'ncmp000')
5 ('pa

4 ('bb', 's')
4 ('bay', 'n')
4 ('barrient', 'n')
4 ('bancrec', 's')
4 ('ayud', 'vmn0000')
4 ('avanz', 'n')
4 ('autop', 'n')
4 ('aument', 'vmp00sm')
4 ('atrap', 'n')
4 ('atent', 's')
4 ('artificial', 'aq0cs0')
4 ('articul', 'ncmp000')
4 ('argument', 'vmip3s0')
4 ('argument', 'n')
4 ('are', 'ncfs000')
4 ('aquell', 'dd0fp0')
4 ('apell', 'ncms000')
4 ('apel', 'ncfs000')
4 ('aparent', 'aq0cs0')
4 ('anunc', 'vmis3s0')
4 ('ampliacion', 'ncfs000')
4 ('american', 'aq0fs0')
4 ('alma', 'ncfs000')
4 ('aliment', 'ncmp000')
4 ('aliment', 'ncfs000')
4 ('advers', 'n')
4 ('adquisicion', 'ncfs000')
4 ('admit', 'vmis3s0')
4 ('adeud', 'n')
4 ('adel', 'rg')
4 ('actu', 'vmn0000')
4 ('actos', 'ncmp000')
4 ('acontec', 's')
4 ('aclar', 'vmis3s0')
4 ('aclar', 's')
4 ('absolut', 'rg')
4 ('abril', 'W')
4 ('abiert', 'aq0fsp')
4 ('abastec', 'ncms000')
3 ('zapat', 'n')
3 ('xxi', 's')
3 ('x', 's')
3 ('weathers', 's')
3 ('washington', 's')
3 ('vuelv', 'vmip3s0')
3 ('voz', 'ncfs000')
3 ('vot', 'vmis3s0')
3 ('viv', 'vmi

3 ('bataz', 'n')
3 ('bast', 'vmip3s0')
3 ('basic', 'rg')
3 ('bas', 'aq0msp')
3 ('barril', 'ncmp000')
3 ('banxic', 'n')
3 ('band', 'ncmp000')
3 ('bancari', 'aq0mp0')
3 ('añad', 'vmn0000')
3 ('avispon', 's')
3 ('avanz', 'aq0fsp')
3 ('avantel', 's')
3 ('autoritar', 'n')
3 ('automotriz', 's')
3 ('aureli', 'n')
3 ('aument', 'vmn0000')
3 ('aument', 'vmip3s0')
3 ('aument', 's')
3 ('atun', 's')
3 ('atlant', 'n')
3 ('atlant', 'aq0ms0')
3 ('atend', 'vmn0000')
3 ('atend', 'n')
3 ('aspect', 'ncms000')
3 ('asever', 'vmis3s0')
3 ('asent', 's')
3 ('asegur', 'vmn0000')
3 ('asegur', 'vmip3p0')
3 ('asegur', 'ncfs000')
3 ('ascens', 'ncms000')
3 ('ascend', 'vmis3s0')
3 ('argentin', 'aq0fs0')
3 ('are', 'ncfp000')
3 ('aquell', 'dd0mp0')
3 ('aprueb', 's')
3 ('aprovech', 'vmn0000')
3 ('aprend', 'vmn0000')
3 ('aplic', 'ncfs000')
3 ('apetit', 'ncms000')
3 ('antigu', 'aq0fs0')
3 ('anticip', 's')
3 ('andres', 's')
3 ('ancian', 'n')
3 ('analisis', 'ncmn000')
3 ('ampli', 'vmp00sm')
3 ('amig', 'ncmp000')
3 ('amenaz'

...

1 ('misteri', 'ncms000')
1 ('mismisim', 'di0ms0')
1 ('mis', 'ncfs000')
1 ('mir', 's')
1 ('mir', 'ncfs000')
1 ('mir', 'ncfp000')
1 ('minut', 'ncms000')
1 ('minut', 'n')
1 ('minuci', 's')
1 ('minuci', 'ncfp000')
1 ('minseit', 'n')
1 ('minoritari', 'aq0mp0')
1 ('minor', 'ncfp000')
1 ('mink', 's')
1 ('minimiz', 's')
1 ('miner', 'ncmp000')
1 ('miner', 'aq0fp0')
1 ('min', 'ncfp000')
1 ('min', 'n')
1 ('mimin', 's')
1 ('milwauke', 's')
1 ('millonews', 's')
1 ('milllon', 's')
1 ('militariz', 's')
1 ('militar', 'n')
1 ('milit', 'ncfs000')
1 ('milit', 'n')
1 ('milit', 'aq0cs0')
1 ('milagr', 'n')
1 ('migratori', 'n')
1 ('miembr', 'ncms000')
1 ('microproces', 's')
1 ('microeconom', 'n')
1 ('micr', 'n')
1 ('michelen', 'n')
1 ('mgr', 's')
1 ('mezcl', 'vmip3s0')
1 ('mey', 's')
1 ('mexiquens', 's')
1 ('mexicali', 's')
1 ('mexic', 'n')
1 ('mex', 's')
1 ('metoxitriptamin', 'n')
1 ('metan', 'n')
1 ('metal', 'aq0ms0')
1 ('messm', 's')
1 ('merm', 's')
1 ('merm', 'n')
1 ('merec', 'vmip3s0')
1 ('merec', 'vmip

1 ('importantisim', 'aq0fs0')
1 ('import', 'vmip3s0')
1 ('implac', 'aq0cs0')
1 ('impid', 'vmis3s0')
1 ('imperi', 'ncms000')
1 ('imperdon', 's')
1 ('imped', 'vmii3p0')
1 ('imped', 'vmic3s0')
1 ('impec', 'aq0cs0')
1 ('impart', 'n')
1 ('impact', 's')
1 ('imcp', 's')
1 ('imagin', 's')
1 ('imag', 'ncfs000')
1 ('ilustr', 'aq0cs0')
1 ('ilog', 'aq0ms0')
1 ('illañez', 's')
1 ('ilegal', 'aq0cs0')
1 ('iguald', 'ncfs000')
1 ('igual', 's')
1 ('ignor', 'vmip3s0')
1 ('ife', 's')
1 ('iempr', 's')
1 ('ido', 'vmp00sm')
1 ('ideologiz', 's')
1 ('identific', 'n')
1 ('identif', 'vmip3s0')
1 ('ideari', 'n')
1 ('ideal', 'ncmp000')
1 ('ibarr', 'n')
1 ('iban', 'vmii3p0')
1 ('huracan', 'ncms000')
1 ('hungar', 'n')
1 ('humani', 's')
1 ('huicoche', 'n')
1 ('hufbau', 's')
1 ('huerfan', 'n')
1 ('huelg', 'ncfs000')
1 ('huejutl', 'n')
1 ('hubbl', 's')
1 ('huapac', 's')
1 ('htm', 's')
1 ('hostil', 's')
1 ('horripil', 's')
1 ('horribl', 'aq0cs0')
1 ('horn', 'ncms000')
1 ('horizont', 'ncms000')
1 ('horc', 'n')
1 ('honrad

1 ('escoci', 'n')
1 ('esclavitud', 's')
1 ('escenari', 'ncms000')
1 ('escenari', 'ncmp000')
1 ('escatim', 's')
1 ('escas', 'rg')
1 ('escarb', 'n')
1 ('escaramuz', 'ncfp000')
1 ('escaramuz', 'n')
1 ('escarabajil', 's')
1 ('escap', 'vmn0000')
1 ('escap', 's')
1 ('escandal', 's')
1 ('escandal', 'n')
1 ('escam', 'n')
1 ('escaler', 'ncfs000')
1 ('erot', 'n')
1 ('erosion', 'n')
1 ('erig', 's')
1 ('eribert', 'n')
1 ('equivoc', 'n')
1 ('equivalent', 'ncms000')
1 ('equival', 'vmip3s0')
1 ('equit', 'n')
1 ('equipar', 's')
1 ('equilibr', 'vmn0000')
1 ('epigram', 'n')
1 ('epifenomen', 'n')
1 ('envuelt', 'n')
1 ('envidi', 's')
1 ('envidi', 'n')
1 ('envi', 'vmn0000')
1 ('envi', 'vmis3s0')
1 ('envi', 'vmip3s0')
1 ('envi', 's')
1 ('envi', 'n')
1 ('envenen', 'n')
1 ('enunci', 'vmip3p0')
1 ('enunci', 'n')
1 ('entusiast', 'n')
1 ('entroniz', 'n')
1 ('entro', 'vmis3s0')
1 ('entrevist', 'vmif3s0')
1 ('entrevist', 's')
1 ('entrevist', 'n')
1 ('entrev', 'vmn0000')
1 ('entrev', 'ncfp000')
1 ('entreg', 'n')
1 

1 ('creador', 'ncmp000')
1 ('cre', 'vmis3p0')
1 ('cre', 'vmg0000')
1 ('cras', 'n')
1 ('crack', 's')
1 ('crac', 's')
1 ('cp', 's')
1 ('coyoacan', 's')
1 ('cox', 's')
1 ('cotiz', 'ncfs000')
1 ('cotiz', 'aq0mpp')
1 ('cotidian', 'aq0fs0')
1 ('cotic', 's')
1 ('costumbr', 'ncfp000')
1 ('costos', 'n')
1 ('costos', 'aq0ms0')
1 ('coster', 'n')
1 ('cost', 'vmis3s0')
1 ('cost', 'ncfp000')
1 ('cosmovision', 's')
1 ('cortin', 's')
1 ('cort', 'vmn0000')
1 ('cort', 'ncfp000')
1 ('cort', 'n')
1 ('corruptel', 'n')
1 ('corroid', 'n')
1 ('corrig', 'n')
1 ('corrient', 'ncfs000')
1 ('corresponsal', 's')
1 ('correspondient', 'aq0cp0')
1 ('correspond', 'vmsp3s0')
1 ('correspond', 'vmip3s0')
1 ('correligionari', 'n')
1 ('correg', 's')
1 ('correg', 'n')
1 ('corredur', 'n')
1 ('correct', 'n')
1 ('corr', 'vmis3s0')
1 ('corr', 'vmip3p0')
1 ('corr', 'n')
1 ('corporal', 'aq0cs0')
1 ('corpor', 'aq0fs0')
1 ('cordiller', 'ncfp000')
1 ('corder', 'n')
1 ('corazon', 'ncms000')
1 ('coraj', 'ncms000')
1 ('coquete', 'n')
1 

1 ('ampli', 'aq0fs0')
1 ('ampli', 'aq0fp0')
1 ('amotin', 's')
1 ('amortigu', 'vmn0000')
1 ('amorrortu', 's')
1 ('amoniac', 'n')
1 ('amig', 'ncms000')
1 ('amig', 'ncfs000')
1 ('americ', 's')
1 ('amenaz', 'vmp00sm')
1 ('amenaz', 'ncfs000')
1 ('amenaz', 'ncfp000')
1 ('amen', 's')
1 ('ambit', 'ncmp000')
1 ('ambigüed', 's')
1 ('ambiental', 'n')
1 ('ambag', 's')
1 ('amarr', 's')
1 ('amarill', 'aq0fs0')
1 ('amaral', 's')
1 ('amant', 'nccs000')
1 ('amad', 'n')
1 ('amabl', 'aq0cp0')
1 ('alzheim', 's')
1 ('alzamient', 'n')
1 ('alvarez', 's')
1 ('alumn', 'n')
1 ('alter', 's')
1 ('alter', 'ncfp000')
1 ('alter', 'n')
1 ('altar', 'ncmp000')
1 ('alt', 'rg')
1 ('alpont', 's')
1 ('almoloy', 'n')
1 ('almacenaj', 's')
1 ('almacen', 'ncms000')
1 ('alleg', 's')
1 ('allain', 's')
1 ('alin', 'vmp00pm')
1 ('alin', 'ncfp000')
1 ('alic', 's')
1 ('ali', 's')
1 ('ali', 'ncmp000')
1 ('alfred', 'n')
1 ('alexei', 's')
1 ('alexand', 's')
1 ('alert', 'ncfs000')
1 ('alent', 'vmn0000')
1 ('alej', 'vmip3s0')
1 ('alej', '